In [ ]:
import csv
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

In [ ]:
# Specify directory that contains files with measures
dir = "/Users/laraverheyen/Projects/Babel3/applications/visual-dialog/measures/data/"

names = joinNodenames(dir)
fcg_questions = fcg_measures("QUESTIONS-INTRODUCED-BY-GRAMMAR.csv")
fcg_answers = fcg_measures("QUESTIONS-SOLVED-BY-GRAMMAR.csv")
#irl_answers = irl_measures("QUESTIONS-SOLVED-BY-MENTAL-SIMULATION.csv")
irl_answers_initial = irl_measures("INITIAL.csv")
irl_answers_discourse = irl_measures("DISCOURSE.csv")
irl_answers_perception = irl_measures("PERCEPTION.csv")
irl_answers_inference = irl_measures("INFERENCE.csv")

#answers = combine_lists([fcg_answers, irl_answers])
answers = combine_lists([fcg_answers, irl_answers_initial, irl_answers_discourse, irl_answers_perception, irl_answers_inference])
plot_lines(fcg_questions, answers)

plot_stackplot(fcg_questions, 'questions', [fcg_answers, irl_answers_initial, irl_answers_discourse, irl_answers_perception, irl_answers_inference], ['answers from language', 'answers initial', 'answers from discourse', 'answers from perception', 'answers from inference'])

#total_answers = combine_lists([fcg_answers, irl_answers])
unsolved_questions = subtract_lists([cumSum(answers), cumSum(fcg_questions)])
plot_pies([cumSum(fcg_answers), cumSum(irl_answers_initial) , cumSum(irl_answers_discourse), cumSum(irl_answers_perception), cumSum(irl_answers_inference), unsolved_questions], ['answers from language', 'answers initial', 'answers from discourse', 'answers from perception', 'answers from inference', 'remaining questions'])

In [ ]:
def plot_pies(values, labels): 

    for i in range(0, len(values[0]), 10): 

        fig, ax = plt.subplots()
        fig.set_size_inches(10.5, 13.5)
        
        vals = [val[i] for val in values]
        
        ax.pie(vals, labels=None,autopct='%1.1f%%',  textprops={'fontsize': 30})
        ax.axis('equal')
        plt.legend(labels=labels,prop={'size': 20},loc='lower left')

        plt.savefig("pie-after-instructions-" + str(i) + ".pdf")
        

In [ ]:
def plot_stackplot(line_numbers, line_label, stack_numbers, stack_labels): 
    
    fig, ax = plt.subplots()
    fig.set_size_inches(18.5, 10.5)
    
    cumline = cumSum(line_numbers)
    vals = [x for x in range(0, len(stack_numbers[0]))]
    total = []
    for number in stack_numbers: 
        total.append(cumSum(number))
    
    ax.stackplot(vals, total, labels=stack_labels, alpha=0.8)

    ax.plot(cumline, label=line_label,color="black", linewidth=1.5)
    
    plt.xticks([])

    ax.set_ylabel('Number of questions')
    ax.set_xlabel('Steps in recipe execution')
    ax.legend(prop={"size":20})
    ax.yaxis.label.set_fontsize(20)
    ax.xaxis.label.set_fontsize(20)
    plt.yticks(fontsize=15)
    
    plt.savefig('measures_stackplot.pdf')

In [ ]:
def plot_lines(questions, answers):

    questions = cumSum(questions)
    answers = cumSum(answers)

    fig, ax = plt.subplots()
    fig.set_size_inches(18.5, 10.5)

    ax.plot(questions, label='Number of questions',color="black", linewidth=1.5)
    ax.plot(answers, label='Number of answers',color="red", linewidth=1.5)
    
    plt.xticks([])
    ax.set_ylabel('Number of questions')
    ax.set_xlabel('Steps in dialog')
    ax.legend(prop={"size":20})
    ax.yaxis.label.set_fontsize(20)
    ax.xaxis.label.set_fontsize(20)
    plt.yticks(fontsize=15)

    plt.savefig('measures_line_plot.pdf')

In [ ]:
## Functions to read measures from files 
## if you give measures from IRL, it will also append fcg measures 0 to it and the other way around.

def fcg_measures(measures_file): 
    fcg_measures_file = os.path.join(dir, measures_file)
    irl_names_file = os.path.join(dir, "irl-nodenames.csv")
    
    fcg_file = open(fcg_measures_file)
    fcg_measures = csv.reader(fcg_file, delimiter=' ')
    irl_file = open(irl_names_file)
    irl_names = csv.reader(irl_file, delimiter=' ')
    
    all_measures = []
    for fcg_measure, irl_name in zip(fcg_measures, irl_names):
        fcg_measure_int = [int(x) for x in fcg_measure]
        all_measures.append(fcg_measure_int)
        irl_measure = [0] * len(irl_name)
        all_measures.append(irl_measure)
    return flatten(all_measures)

def irl_measures(measures_file): 
    fcg_names_file = os.path.join(dir, "fcg-nodenames.csv")
    irl_measures_file = os.path.join(dir, measures_file)
    
    fcg_file = open(fcg_names_file)
    fcg_names = csv.reader(fcg_file, delimiter=' ')
    irl_file = open(irl_measures_file)
    irl_measures = csv.reader(irl_file, delimiter=' ')
    
    all_measures = []
    for fcg_name, irl_measure in zip(fcg_names, irl_measures):
        fcg_measure = [0] * len(fcg_name)
        all_measures.append(fcg_measure)
        irl_measure_int = [int(x) for x in irl_measure]
        all_measures.append(irl_measure_int)
    return flatten(all_measures)

# Put together nodenames from FCG and IRL
def joinNodenames(dir):
    fcg_names_file = os.path.join(dir, "fcg-nodenames.csv")
    irl_names_file = os.path.join(dir, "irl-nodenames.csv")
    
    fcg_file = open(fcg_names_file)
    fcg_names = csv.reader(fcg_file, delimiter=' ')
    irl_file = open(irl_names_file)
    irl_names = csv.reader(irl_file, delimiter=' ')
    
    all_nodenames = []
    for fcg_name, irl_name in zip(fcg_names, irl_names):
        all_nodenames.append(fcg_name)
        all_nodenames.append(irl_name)
        
    return flatten(all_nodenames)

In [ ]:
## UTILS
def cumSum(s):
   sm=0
   cum_list=[]
   for i in s:
      sm=sm+i
      cum_list.append(sm)
   return cum_list

def flatten(l):
    return [item for sublist in l for item in sublist]

def combine_lists(lists):
    all_lists = [0] * len(lists[0])
    for list in lists: 
        all_lists = [sum(i) for i in zip(list, all_lists)]
    return all_lists

def subtract_lists(lists):
    all_lists = [0] * len(lists[0])
    for list in lists:
        all_lists = [(i - j) for i,j in zip(list, all_lists)]
    return all_lists

In [ ]:
plot_stacked_bars()

In [ ]:
plot_remaining_questions()

In [ ]:
def norm_cumulative_list(cum_lst, greatest_el): 
    norm_cum_list = []
    for el in cum_lst: 
        norm_cum_list.append(el/greatest_el)
    return norm_cum_list

In [ ]:
### these pies 

data_dir = "data-for-python-whole/"
irl_filenames = ['irl-nodenames.csv','RECORD-ANCHORING-MICRO.csv', 'RECORD-QUESTIONS-IRL.csv', 'RECORD-SET-SLOTS.csv' , 'RECORD-SLOTS.csv']
fcg_filenames = ['fcg-nodenames.csv','RECORD-VARIABLES-BOUND-BY-GRAMMAR-MICRO.csv', 'RECORD-VARIABLES-INTRODUCED-BY-GRAMMAR-MICRO.csv','RECORD-ACCESSIBLE-ENTITIES-BOUND-BY-GRAMMAR-MICRO.csv']
dicts = files_to_dict(data_dir, irl_filenames, fcg_filenames)
dict_fcg = dicts[0]
dict_irl = dicts[1]
dict_irl = fix_length_aes(dict_irl)
final_d = combine_irl_and_fcg(dict_fcg,dict_irl)

flattened_dictionary = {}

for key in final_d.keys():
    regular_list = final_d[key]
    lst = [item for sublist in regular_list for item in sublist]
    flattened_dictionary[key] = lst 

node_names = flattened_dictionary['nodenames']
introduced_vars = cumSum([int(item) for item in flattened_dictionary['introduced_vars']])
bound_vars_fcg = cumSum([int(item) for item in flattened_dictionary['bound_vars_fcg']])
bound_aes_fcg = cumSum([int(item) for item in flattened_dictionary['bound_aes_fcg']])
bound_vars_irl = cumSum([int(item) for item in flattened_dictionary['bound_vars_irl']])
bound_slots_irl = cumSum([int(item) for item in flattened_dictionary['bound_slots_irl']])

lst = []
for i in range(0,len(bound_vars_fcg)):
    lst.append(i)

fig, ax = plt.subplots()
#fig.set_size_inches(5,5)
plt.figure(figsize = (5, 15))

explode = (0, 0, 0, 0, 0.1) 
x = [61,74,87,96,111,117,131,len(introduced_vars)-1]
#i = 0
def update(i): 
#for x in [61,111,len(introduced_vars)-1]:
    ax.clear()
    ax.axis('equal')
    total = sum(introduced_vars[:x[i]])
    bound_vars = {'Answers from language': sum(bound_vars_fcg[:x[i]]), 
                 'Answers from mental simulation': sum(bound_vars_irl[:x[i]]), 
                 'Answers from ontology' : sum(bound_slots_irl[:x[i]]), 
                 'Answers from discourse model': sum(bound_aes_fcg[:x[i]]),
                 'Remaining questions': total - sum(bound_vars_fcg[:x[i]]) - sum(bound_vars_irl[:x[i]]) - sum(bound_slots_irl[:x[i]]) - sum(bound_aes_fcg[:x[i]])}
    ax.pie(bound_vars.values(), explode=explode, labels=None,autopct='%1.1f%%', textprops={'fontsize': 10})
    
    ax.set_title("{}".format(recipe[i]))
ani = FuncAnimation(fig, update, frames=range(8),  repeat=False)

f = "questions-solved-compared-to-questions-introduced.gif" 
ani.save(f, fps=1)


In [ ]:
%matplotlib notebook
animate()

In [ ]:
plt.ioff()

In [ ]:
### these pies are cumulative

data_dir = "data-for-python-whole/"
irl_filenames = ['irl-nodenames.csv','RECORD-ANCHORING-MICRO.csv', 'RECORD-QUESTIONS-IRL.csv', 'RECORD-SET-SLOTS.csv' , 'RECORD-SLOTS.csv']
fcg_filenames = ['fcg-nodenames.csv','RECORD-VARIABLES-BOUND-BY-GRAMMAR-MICRO.csv', 'RECORD-VARIABLES-INTRODUCED-BY-GRAMMAR-MICRO.csv','RECORD-ACCESSIBLE-ENTITIES-BOUND-BY-GRAMMAR-MICRO.csv']
dicts = files_to_dict(data_dir, irl_filenames, fcg_filenames)
dict_fcg = dicts[0]
dict_irl = dicts[1]
dict_irl = fix_length_aes(dict_irl)
final_d = combine_irl_and_fcg(dict_fcg,dict_irl)

flattened_dictionary = {}

for key in final_d.keys():
    regular_list = final_d[key]
    lst = [item for sublist in regular_list for item in sublist]
    flattened_dictionary[key] = lst 

node_names = flattened_dictionary['nodenames']
introduced_vars = cumSum([int(item) for item in flattened_dictionary['introduced_vars']])
bound_vars_fcg = cumSum([int(item) for item in flattened_dictionary['bound_vars_fcg']])
bound_aes_fcg = cumSum([int(item) for item in flattened_dictionary['bound_aes_fcg']])
bound_vars_irl = cumSum([int(item) for item in flattened_dictionary['bound_vars_irl']])
bound_slots_irl = cumSum([int(item) for item in flattened_dictionary['bound_slots_irl']])


bound_vars = {'Answers from language': 0, 
                 'Answers from mental simulation': 0, 
                 'Answers from ontology' :0, 
                 'Answers from discourse model': 0,
                 'Remaining questions': 0}
    

lst = []
for i in range(0,len(bound_vars_fcg)):
    lst.append(i)

fig, ax = plt.subplots()
#fig.set_size_inches(5,5)
plt.figure(figsize = (5, 15))

explode = (0, 0, 0, 0, 0.1) 
x = [61,74,87,96,111,117,131,len(introduced_vars)]
#i = 0

def update(i): 
#for x in [61,111,len(introduced_vars)-1]:
    ax.clear()
    #ax.axis('equal')
    #total = sum(introduced_vars[:x[i]])
    total = sum(introduced_vars[:x[i]])
    radius = sum(introduced_vars[:x[i]]) / sum(introduced_vars)
    print(radius)
    bound_vars = {'Answers from language': sum(bound_vars_fcg[:x[i]]), 
                 'Answers from mental simulation': sum(bound_vars_irl[:x[i]]), 
                'Answers from ontology' : sum(bound_slots_irl[:x[i]]), 
                 'Answers from discourse model': sum(bound_aes_fcg[:x[i]]),
                 'Remaining questions': total - sum(bound_vars_fcg[:x[i]]) - sum(bound_vars_irl[:x[i]]) -sum(bound_slots_irl[:x[i]]) - sum(bound_aes_fcg[:x[i]])}
    
    ax.pie(bound_vars.values(), explode=None, labels=None,autopct='%1.1f%%', radius=radius, textprops={'fontsize': 10})
    
    ax.set_title("{}".format(recipe[i]))
    #if x == 61:
    ax.legend(labels = bound_vars.keys(), loc="upper left")

    print("{}".format(recipe[i]))

#plt.legend(labels=bound_vars.keys(),loc='upper left')

ani = FuncAnimation(fig, update, frames=range(8),  repeat=False)
f = "test-cumulative.gif" 
ani.save(f, fps=0.9)


In [ ]:
### these pies 

data_dir = "data-for-python-whole/"
irl_filenames = ['irl-nodenames.csv','RECORD-ANCHORING-MICRO.csv', 'RECORD-QUESTIONS-IRL.csv', 'RECORD-SET-SLOTS.csv' , 'RECORD-SLOTS.csv']
fcg_filenames = ['fcg-nodenames.csv','RECORD-VARIABLES-BOUND-BY-GRAMMAR-MICRO.csv', 'RECORD-VARIABLES-INTRODUCED-BY-GRAMMAR-MICRO.csv','RECORD-ACCESSIBLE-ENTITIES-BOUND-BY-GRAMMAR-MICRO.csv']
dicts = files_to_dict(data_dir, irl_filenames, fcg_filenames)
dict_fcg = dicts[0]
dict_irl = dicts[1]
dict_irl = fix_length_aes(dict_irl)
final_d = combine_irl_and_fcg(dict_fcg,dict_irl)

flattened_dictionary = {}

for key in final_d.keys():
    regular_list = final_d[key]
    lst = [item for sublist in regular_list for item in sublist]
    flattened_dictionary[key] = lst 

node_names = flattened_dictionary['nodenames']
introduced_vars = [int(item) for item in flattened_dictionary['introduced_vars']]
bound_vars_fcg = [int(item) for item in flattened_dictionary['bound_vars_fcg']]
bound_aes_fcg = [int(item) for item in flattened_dictionary['bound_aes_fcg']]
bound_vars_irl = [int(item) for item in flattened_dictionary['bound_vars_irl']]
bound_slots_irl = [int(item) for item in flattened_dictionary['bound_slots_irl']]



bound_vars = {'Answers from language': 0, 
              'Answers from mental simulation': 0, 
                 'Answers from ontology' :0, 
                 'Answers from discourse model': 0,
                 'Remaining questions': 0}
    

lst = []
for i in range(0,len(bound_vars_fcg)):
    lst.append(i)

fig, ax = plt.subplots()
plt.figure(figsize = (5, 15))

explode = (0, 0, 0, 0, 0.1) 
x = [61,74,87,96,111,117,131,len(introduced_vars)]
#i = 0

def update(i): 
    ax.clear()
    print(x[i-1])
    total = sum(introduced_vars[x[i-1]:x[i]])
    bound_vars = {'Answers from language': sum(bound_vars_fcg[x[i-1]:x[i]]), 
                  'Answers from mental simulation': sum(bound_vars_irl[x[i-1]:x[i]]), 
                  'Answers from ontology' : sum(bound_slots_irl[x[i-1]:x[i]]), 
                  'Answers from discourse model': sum(bound_aes_fcg[x[i-1]:x[i]]),
                  'Remaining questions': total - sum(bound_vars_fcg[x[i-1]:x[i]]) - sum(bound_vars_irl[x[i-1]:x[i]]) -sum(bound_slots_irl[x[i-1]:x[i]]) - sum(bound_aes_fcg[x[i-1]:x[i]])}
    
    ax.pie(bound_vars.values(), explode=None, labels=None,autopct='%1.1f%%', radius=1, textprops={'fontsize': 10})
    
    ax.set_title("{}".format(recipe[i]))
   # ax.legend(labels = bound_vars.keys(), loc="lower right")

    print("{}".format(recipe[i]))

#plt.legend(labels=bound_vars.keys(),loc='lower right')

ani = FuncAnimation(fig, update, frames=range(8),  repeat=False)
f = "test.gif" 
ani.save(f, fps=0.9)


In [ ]:
### these pies are cumulative

data_dir = "data-for-python/"
irl_filenames = ['irl-nodenames.csv','RECORD-ANCHORING-MICRO.csv', 'RECORD-QUESTIONS-IRL.csv', 'RECORD-SET-SLOTS.csv' , 'RECORD-SLOTS.csv']
fcg_filenames = ['fcg-nodenames.csv','RECORD-VARIABLES-BOUND-BY-GRAMMAR-MICRO.csv', 'RECORD-VARIABLES-INTRODUCED-BY-GRAMMAR-MICRO.csv','RECORD-ACCESSIBLE-ENTITIES-BOUND-BY-GRAMMAR-MICRO.csv']
dicts = files_to_dict(data_dir, irl_filenames, fcg_filenames)
dict_fcg = dicts[0]
dict_irl = dicts[1]
dict_irl = fix_length_aes(dict_irl)
final_d = combine_irl_and_fcg(dict_fcg,dict_irl)

flattened_dictionary = {}

for key in final_d.keys():
    regular_list = final_d[key]
    lst = [item for sublist in regular_list for item in sublist]
    flattened_dictionary[key] = lst 

node_names = flattened_dictionary['nodenames']
print(node_names)
introduced_vars = cumSum([int(item) for item in flattened_dictionary['introduced_vars']])
bound_vars_fcg = cumSum([int(item) for item in flattened_dictionary['bound_vars_fcg']])
bound_aes_fcg = cumSum([int(item) for item in flattened_dictionary['bound_aes_fcg']])
bound_vars_irl = cumSum([int(item) for item in flattened_dictionary['bound_vars_irl']])
bound_slots_irl = cumSum([int(item) for item in flattened_dictionary['bound_slots_irl']])


bound_vars = {'Answers from language': 0, 
                 'Answers from mental simulation': 0, 
                 'Answers from ontology' :0, 
                 'Answers from discourse model': 0,
                 'Remaining questions': 0}
    

lst = []
for i in range(0,len(bound_vars_fcg)):
    lst.append(i)


recipe = ["Ingredients", #226 grams butter, room temperature. 116 grams sugar. \n4 grams vanilla extract, 4 grams almond extract, 340 grams flour, \n112 grams almond flour, and 29 grams powdered sugar", 
"Beat the butter and the sugar together until light and fluffy.",
"Add the vanilla and almond extracts and mix.",
"Add the flour and the almond flour. ",
"Mix thoroughly.",
"Take generous tablespoons of the dough and roll it into a small ball, \nabout an inch in diameter, and then shape it into a crescent shape.",
"Place onto a parchment paper lined baking sheet.",
"Bake at 175 degrees Celsius for 15 - 20 minutes. ",
"Dust with powdered sugar."]
 
x = [61,74,87,96,111,117,131,len(introduced_vars)]
#i = 0

for x in [61,74,87,96,111,117,131,len(introduced_vars)]:
#for x in [61,111,len(introduced_vars)-1]:
    fig, ax = plt.subplots()
    fig.set_size_inches(5,5)
    #plt.figure(figsize = (15, 15))
    explode = (0, 0, 0, 0, 0.1)
    #ax.clear()
    #ax.axis('equal')
    #total = sum(introduced_vars[:x[i]])
    total = sum(introduced_vars[:x])
    radius = sum(introduced_vars[:x]) / sum(introduced_vars)
    print(radius)
    bound_vars = {'Answers from language': sum(bound_vars_fcg[:x]), 
                 'Answers from mental simulation': sum(bound_vars_irl[:x]), 
                'Answers from ontology' : sum(bound_slots_irl[:x]), 
                 'Answers from discourse model': sum(bound_aes_fcg[:x]),
                 'Remaining questions': total - sum(bound_vars_fcg[:x]) - sum(bound_vars_irl[:x]) - sum(bound_slots_irl[:x]) - sum(bound_aes_fcg[:x])}
    
    if x == 61:
        pctdistance = 1.5
    else:
        pctdistance = 0.6
    
    ax.pie(bound_vars.values(), explode=None, labels=None,  pctdistance =pctdistance, autopct='%1.1f%%', radius=radius, textprops={'fontsize': 10})
    
    if x == 61: 
        ax.legend(labels=bound_vars.keys(),prop={'size': 10},loc='upper left')
        
    plt.savefig('pie-chart-' + str(x) + '.pdf' , bbox_inches='tight')
    

#plt.legend(labels=bound_vars.keys(),loc='upper left')
    plt.show()
#ani = FuncAnimation(fig, update, frames=range(8),  repeat=False)
#f = "test-cumulative.gif" 
#ani.save(f, fps=0.9)
